# HW3 : Math for Robotics

Author: Ruffin White  
Course: CSE291  
Date: Feb 23 2018

In [ ]:
# Make plots inline
# %matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('pdf', 'svg')
# set_matplotlib_formats('png', 'pdf')
set_matplotlib_formats('png')

# import modules for plotting and data analysis
import matplotlib.pyplot as plt

## 1. 

> Consider the following differential equation over the interval $(0,1]$:

$$
\frac{dy}{dx} = \frac{1}{x^2(1-y)}
$$

with $y(1)=-1$

* Obtain an exact analytical solution to the equation. In the following solve for $y(0)$ even though in theory the equation is not defined for $x=0$.
* Implement and use Eulers method to solve the differential equation numerically. Use a step size of $0.05$. How accurate is your numerical solution?
* Implement and use a fourth-order Runge-Kutta method to solve the differential equation numerically. Again, use a step size of $0.05$. Again, how accurate is your numerical solution?
* Implement and use a Richardson extrapolation to solve the equation, again with a step size of $0.05$. How accuracy is your solution compared to the analytical solution?

In [ ]:
import re
import numpy as np

from scipy.integrate import ode
from scipy.integrate._ode import IntegratorBase

In [ ]:
class MyIntegratorBase(IntegratorBase):
    def run(self, f, jac, y0, t0, t1, f_params, jac_params):
        y1, t, istate = self.runner(f, jac, y0, t0, t1, f_params, jac_params)
        self.istate = istate
        if istate < 0:
            unexpected_istate_msg = 'Unexpected istate={:d}'.format(istate)
            warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,
                          self.messages.get(istate, unexpected_istate_msg)))
            self.success = 0
        return y1, t

def find_integrator(name):
    for cl in integrator_classes:
        if re.match(name, cl.__name__, re.I):
            return cl
    return None

class my_ode(ode):
    def set_integrator(self, name, **integrator_params):
        """
        Set integrator by name.
        Parameters
        ----------
        name : str
            Name of the integrator.
        integrator_params
            Additional parameters for the integrator.
        """
        integrator = find_integrator(name)
        if integrator is None:
            raise ValueError('No integrator name match with %r or is not available.' % name)
        else:
            self._integrator = integrator(**integrator_params)
            if not len(self._y):
                self.t = 0.0
                self._y = np.array([0.0], self._integrator.scalar)
            self._integrator.reset(len(self._y), self.jac is not None)
        return self

In [ ]:
class euler(MyIntegratorBase):
    def __init__(self, **integrator_params):
        self.success = 1
        pass
    
    def runner(self, f, jac, y0, t0, t1, f_params, jac_params):
        y1 = y0 + jac(t0, y0, *jac_params) * (t1 - t0)
        
        istate = 0
        return y1, t1, istate

class rungekutta4(MyIntegratorBase):
    def __init__(self, **integrator_params):
        self.success = 1
        pass
    
    def runner(self, f, jac, y0, t0, t1, f_params, jac_params):
        h = t1-t0
        k_1 = jac((t0        ), (y0            ), *jac_params)
        k_2 = jac((t0 + 0.5*h), (y0 + 0.5*h*k_1), *jac_params)
        k_3 = jac((t0 + 0.5*h), (y0 + 0.5*h*k_2), *jac_params)
        k_4 = jac((t0 +     h), (y0 +     h*k_3), *jac_params)
        y1 = y0 + (h/6)*(k_1 + 2*k_2 + 2*k_3 + k_4)
        
        istate = 0
        return y1, t1, istate

class richardson(MyIntegratorBase):
    def __init__(self, **integrator_params):
        self.success = 1
        pass
    
    def runner(self, f, jac, y0, t0, t1, f_params, jac_params):
        # richardson extrapolation  
        # http://www.math.ubc.ca/~feldman/m256/richard.pdf
        istate = 0
        return y1, t1, istate

integrator_classes = [euler, rungekutta4, richardson]

In [ ]:
# https://www.khanacademy.org/math/ap-calculus-bc/bc-diff-equations/bc-eulers-method/v/eulers-method

# y0, t0 = 1 , 0
y0, t0 = -1 , 1

def f(t, y, arg1):
#     return np.e**t
    return 1 / (t * (y - 1))
def jac(t, y, arg1):
#     return y
    return 1 / (t**2 * (1 - y))

# t1 = 1
t1 = 0

# dt = 0.0005
dt = -0.0005

In [ ]:
r = my_ode(f, jac).set_integrator(name='euler')
r.set_initial_value(y0, t0).set_f_params(None).set_jac_params(None)

# while r.successful() and r.t < t1:
while r.successful() and r.t > t1:
    t = r.t+dt
    y = r.integrate(t)

print("t: {0}, y: {1}".format(t, y))

In [ ]:
r = my_ode(f, jac).set_integrator(name='rungekutta4')
r.set_initial_value(y0, t0).set_f_params(None).set_jac_params(None)

# while r.successful() and r.t < t1:
while r.successful() and r.t > t1:
    t = r.t+dt
    y = r.integrate(t)

print("t: {0}, y: {1}".format(t, y))

In [ ]:
r = ode(f, jac).set_integrator(name='dopri5', method='adams', nsteps=10e4)
r.set_initial_value(y0, t0).set_f_params(None).set_jac_params(None)

while r.successful() and r.t < t1:
    t = r.t+dt
    y = r.integrate(t)

print("t: {0}, y: {1}".format(t, y))

## 2.

> Consider a predator-prey dynamics such as the simple Lotka-Volterra model

$$
x' = f(x)
$$

$$
x = 
\begin{pmatrix}
  x_1 \\
  x_2 \\
\end{pmatrix} =
\begin{pmatrix}
  Prey polution \\
  Predator Polution \\
\end{pmatrix}    
$$

$$
f(x) = 
\begin{pmatrix}
  (b-px_2)x_1 \\
  (rx_1 - d)x_2 \\
\end{pmatrix}   
$$

> Without predators, the prey population increases (exponentially) without bound, whereas without prey, the predator population diminishes (exponentially) to zero. The nonlinear interaction, with predators eating prey, tends to diminish the prey population and increase the predator population. Use your Runga-Kutta to solve this system, with the values $b = p = r = d = 1$, $x_1(0) = 0.3$, and $x_2(0) = 0.2$.

In [ ]:
# http://scipy-cookbook.readthedocs.io/items/LoktaVolterraTutorial.html
y0, t0 = np.array([[0.3], [0.2]]), 0

b1 = 1
p1 = 1
r1 = 1
d1 = 1

def f(t, y, arg1):
    return 'foo'

def jac(t, y, b1, p1, r1, d1):
    y = np.array([(b1 - p1*y[1])*y[0],
                  (r1*y[0] - d1)*y[1]])
    return y

t1 = 15
dt = 0.0005

In [ ]:
r = my_ode(f, jac).set_integrator(name='rungekutta4')
r.set_initial_value(y0, t0).set_f_params(None).set_jac_params(b1, p1, r1, d1)

ts = np.array([1])
ys = np.empty([2,1])

while r.successful() and r.t < t1:
    t = r.t+dt
    y = r.integrate(t)
    ts = np.append(ts, t)
    ys = np.append(ys, y, axis=1)

In [ ]:
plt.plot(ts, ys.T)
plt.show()

## 3.

> We have multiple robots that can generate point clouds such as those coming from a Kinect camera. In many cases we want to use the robots to detect objects in its enviroment. We provide three data files:

(a) Empty-Table.txt which containts a data for an empty table  
(b) Cluttered-Table.txt contains point cloud for a cluttered table  
(c) Hallway.txt  

> Each file has the point cloud file in a format with each line contains $x_i y_i z_i$

* Provide a method to estimate the plane parameter for the table. Test it both with the empty and cluttered table. Describe how you filter out the data from the objects. You have to be able to estimate the table parameters in the presence of clutter.
* Describe and show how the method can be generalized to extract all the dominant planes in a relatively empty hallway.

In [ ]:
%matplotlib notebook
# %matplotlib qt
# %matplotlib inline

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

from sklearn import linear_model

from IPython.display import HTML

In [ ]:
def read_pcd(path):
    df = pd.read_csv(path, sep=' ', skiprows=11, names=['x', 'y', 'z'])
    df = df[(df.x != 0) & (df.y != 0) & (df.z != 0)]
    df = df.dropna()
    df['c'] = 0
    df = df.reset_index(drop=True)
    return df

In [ ]:
def plot_3d(df, animated=False, stride=100, ransacs=None):
    # Create a figure and a 3D Axes
    fig = plt.figure(figsize=[8,6])
    ax = Axes3D(fig)
    df = df.iloc[::stride, :]

    def init():
        ax.scatter(xs=df['x'],
                   ys=df['y'],
                   zs=df['z'],
                   c=plt.cm.tab10(df['c']),
                   marker='o')
        ax.set_xlabel('X Axis')
        ax.set_ylabel('Y Axis')
        ax.set_zlabel('Z Axis')
        
        if ransacs:
            for i, ransac in enumerate(ransacs):        
                a = ransac.estimator_.coef_[0,0]
                b = ransac.estimator_.coef_[0,1]
                c = -1
                d = ransac.estimator_.intercept_[0]

                xx, yy = np.meshgrid(np.linspace(-1,1,10), np.linspace(-1,1,10))
                z = (-a * xx - b * yy - d) / c
                ax.plot_surface(xx, yy, z, alpha=0.2, color=plt.cm.tab10(i+1))
        
        return fig,
    
    def animate(i):
        ax.view_init(elev=10., azim=i)
        return fig,
    
    if animated:
        anim = animation.FuncAnimation(fig, animate, init_func=init,
                                       frames=360, interval=20, blit=True)
        video = anim.to_html5_video()
        return video
    else:
        init()
        return fig

In [ ]:
def find_plane(df, residual_threshold=0.1):
    X = df[['x', 'y']].as_matrix()
    y = df[['z']].as_matrix()
    ransac = linear_model.RANSACRegressor(residual_threshold=residual_threshold)
    ransac.fit(X, y)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
#     print("ransac.estimator_.coef_: ", ransac.estimator_.coef_)
#     print("ransac.estimator_.intercept_: ", ransac.estimator_.intercept_)
    return ransac

In [ ]:
# http://scikit-learn.org/stable/modules/clustering.html
# http://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html
from sklearn.cluster import DBSCAN
def find_cluster(df, i, eps=0.1):
    df_cluster = df.copy()
    X = df_cluster[['x', 'y', 'z']].as_matrix()
    db = DBSCAN(eps=0.10, min_samples=100, n_jobs=-1, leaf_size=300).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]
        class_member_mask = (labels == k)
        df_cluster.loc[class_member_mask & core_samples_mask, 'c'] = i + k
    return df_cluster

In [ ]:
def find_planes(df, residual_threshold=0.1, iterations=3, cluster=True):
#     color_idx = np.linspace(0, 1, iterations + 1)
    df_planes = df.copy()
    df_iter = df.copy()
    ransacs = []
    
    for i in range(1, iterations+1):
        ransac = find_plane(
            df=df_iter,
            residual_threshold=residual_threshold)
        ransacs.append(ransac)
        inlier_mask = ransac.inlier_mask_
        outlier_mask = np.logical_not(inlier_mask)
        
        print("Plane Equation #{0}: {1} * x + {2} * y - 1 * z + {3} = 0".format(
            i,
            ransac.estimator_.coef_[0,0],
            ransac.estimator_.coef_[0,1],
            ransac.estimator_.intercept_[0]))
        
        if cluster:
            df_cluster = df_iter[inlier_mask]
            df_cluster = find_cluster(df_cluster, i, eps=residual_threshold)
            j = df_cluster['c'].value_counts().idxmax()
            cluster_mask = df_cluster['c'] == j
            df_cluster.loc[ cluster_mask, 'c'] = i
            df_cluster.loc[~cluster_mask, 'c'] = 0

            df_iter.loc[df_cluster.index] = df_cluster
            df_planes.iloc[df_iter.index] = df_iter

            next_mask = df_iter['c'] == 0
            df_iter = df_iter[next_mask]
        else:
            inlier_mask = ransac.inlier_mask_
            outlier_mask = np.logical_not(inlier_mask)

            df_iter.loc[inlier_mask, 'c'] = i+1
            df_planes.iloc[df_iter.index] = df_iter
            df_iter = df_iter[outlier_mask]
    return df_planes, ransacs

In [ ]:
empty2_path = 'data/Empty2.pcd'
empty2_df = read_pcd(empty2_path)
hallway02c_path = 'data/Hallway02c.pcd'
hallway02c_df = read_pcd(hallway02c_path)
tableWithObjects2_path = 'data/TableWithObjects2.pcd'
tableWithObjects2_df = read_pcd(tableWithObjects2_path)

In [ ]:
empty2_planes_df, ransacs = find_planes(df=empty2_df, residual_threshold=0.1, iterations=3)

In [ ]:
# empty2_video = plot_3d(empty2_planes_df, animated=True, ransacs=ransacs)
# HTML(empty2_video)
fig = plot_3d(empty2_planes_df, ransacs=ransacs)
plt.show()

In [ ]:
hallway02c_planes_df, ransacs = find_planes(df=hallway02c_df, residual_threshold=0.5, iterations=2)

In [ ]:
# hallway02c_video = plot_3d(hallway02c_planes_df, animated=True, ransacs=ransacs)
# HTML(hallway02c_video)
fig = plot_3d(hallway02c_planes_df, ransacs=ransacs)
plt.show()

In [ ]:
tableWithObjects2_planes_df, ransacs = find_planes(df=tableWithObjects2_df, residual_threshold=0.05, iterations=3)

In [ ]:
# tableWithObjects2_video = plot_3d(tableWithObjects2_planes_df, animated=True, stride=10, ransacs=ransacs)
# HTML(tableWithObjects2_video)
fig = plot_3d(tableWithObjects2_planes_df, stride=10, ransacs=ransacs)
plt.show()